# Анализ динамики зарплат в России (2000-2024)

## Описание проекта

Данный проект анализирует динамику среднемесячной номинальной начисленной заработной платы работников организаций по видам экономической деятельности в Российской Федерации за период с 2000 по 2024 год с учетом влияния инфляции.

**Источники данных:**
- Росстат: данные о зарплатах по отраслям
- ЦБ РФ: данные об уровне инфляции

**Анализируемые отрасли:**
1. Всего по экономике
2. Финансовая деятельность  
3. Образование

## 1. Загрузка и подготовка данных

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Настройка стиля графиков
plt.style.use('seaborn-v0_8')
sns.set_palette('deep')

# Загружаем подготовленные данные
salary_data = pd.read_csv('../data/salary_analysis_data.csv')

print("Данные загружены успешно!")
print(f"Период анализа: {salary_data['Год'].min()} - {salary_data['Год'].max()}")
print(f"Анализируемые отрасли: {list(salary_data['Отрасль'].unique())}")
print(f"Всего записей: {len(salary_data)}")

# Просмотр первых строк
salary_data.head()

## 2. Анализ номинальных зарплат

In [ ]:
# Статистика по номинальным зарплатам
print("=== АНАЛИЗ НОМИНАЛЬНЫХ ЗАРПЛАТ ===")
print()

for sector in salary_data['Отрасль'].unique():
    sector_data = salary_data[salary_data['Отрасль'] == sector].copy()
    
    start_salary = sector_data[sector_data['Год'] == 2000]['Зарплата'].iloc[0]
    end_salary = sector_data[sector_data['Год'] == 2024]['Зарплата'].iloc[0]
    growth_multiple = end_salary / start_salary
    avg_growth = (growth_multiple ** (1/24) - 1) * 100
    
    print(f"{sector}:")
    print(f"  2000 год: {start_salary:,.1f} руб")
    print(f"  2024 год: {end_salary:,.1f} руб") 
    print(f"  Рост в {growth_multiple:.1f} раз")
    print(f"  Среднегодовой рост: {avg_growth:.1f}%")
    print()

# Визуализация динамики номинальных зарплат
plt.figure(figsize=(12, 8))
for sector in salary_data['Отрасль'].unique():
    sector_data = salary_data[salary_data['Отрасль'] == sector]
    plt.plot(sector_data['Год'], sector_data['Зарплата'], 
             marker='o', linewidth=3, markersize=6, label=sector)

plt.title('Динамика номинальных зарплат по отраслям (2000-2024)', fontsize=16, fontweight='bold')
plt.xlabel('Год', fontsize=14)
plt.ylabel('Зарплата, руб', fontsize=14)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(fontsize=12)
plt.yscale('log')
plt.tight_layout()
plt.show()

## 3. Анализ инфляции

In [ ]:
# Анализ инфляции
inflation_data = salary_data.drop_duplicates(subset=['Год'])[['Год', 'Инфляция', 'Кумулятивная_инфляция']].sort_values('Год')

print("=== АНАЛИЗ ИНФЛЯЦИИ ===")
print()

avg_inflation = inflation_data['Инфляция'].mean()
max_inflation_row = inflation_data.loc[inflation_data['Инфляция'].idxmax()]
min_inflation_row = inflation_data.loc[inflation_data['Инфляция'].idxmin()]
cumulative_inflation_2024 = inflation_data[inflation_data['Год'] == 2024]['Кумулятивная_инфляция'].iloc[0]

print(f"Средний уровень инфляции за период: {avg_inflation:.1f}%")
print(f"Максимальная инфляция: {max_inflation_row['Инфляция']:.1f}% в {max_inflation_row['Год']} году")
print(f"Минимальная инфляция: {min_inflation_row['Инфляция']:.1f}% в {min_inflation_row['Год']} году")
print(f"Кумулятивная инфляция 2000-2024: {(cumulative_inflation_2024-1)*100:.0f}%")

# Годы высокой инфляции
high_inflation_years = inflation_data[inflation_data['Инфляция'] > 10]['Год'].tolist()
print(f"Годы высокой инфляции (>10%): {high_inflation_years}")

# График инфляции
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))

# Годовая инфляция
bars = ax1.bar(inflation_data['Год'], inflation_data['Инфляция'], 
               color='red', alpha=0.7, edgecolor='darkred')
ax1.axhline(y=10, color='black', linestyle='--', linewidth=2, label='Высокая инфляция (>10%)')
ax1.set_title('Уровень инфляции в России (2000-2024)', fontsize=16, fontweight='bold')
ax1.set_xlabel('Год', fontsize=14)
ax1.set_ylabel('Инфляция, %', fontsize=14)
ax1.grid(True, linestyle='--', alpha=0.7, axis='y')
ax1.legend(fontsize=12)

# Выделяем годы высокой инфляции
for bar, year in zip(bars, inflation_data['Год']):
    if year in high_inflation_years:
        bar.set_color('darkred')

# Кумулятивная инфляция
ax2.plot(inflation_data['Год'], inflation_data['Кумулятивная_инфляция'], 
         marker='o', linewidth=3, markersize=6, color='darkred')
ax2.set_title('Кумулятивная инфляция (базовый год - 2000)', fontsize=16, fontweight='bold')
ax2.set_xlabel('Год', fontsize=14)
ax2.set_ylabel('Кумулятивный коэффициент', fontsize=14)
ax2.grid(True, linestyle='--', alpha=0.7)

plt.tight_layout()
plt.show()

## 4. Анализ реальных зарплат

In [ ]:
# Статистика по реальным зарплатам
print("=== АНАЛИЗ РЕАЛЬНЫХ ЗАРПЛАТ (в ценах 2000 года) ===")
print()

for sector in salary_data['Отрасль'].unique():
    sector_data = salary_data[salary_data['Отрасль'] == sector].copy()
    
    start_real_salary = sector_data[sector_data['Год'] == 2000]['Реальная_зарплата'].iloc[0]
    end_real_salary = sector_data[sector_data['Год'] == 2024]['Реальная_зарплата'].iloc[0]
    real_growth_multiple = end_real_salary / start_real_salary
    real_avg_growth = (real_growth_multiple ** (1/24) - 1) * 100
    
    print(f"{sector}:")
    print(f"  2000 год: {start_real_salary:,.1f} руб")
    print(f"  2024 год: {end_real_salary:,.1f} руб")
    print(f"  Реальный рост в {real_growth_multiple:.1f} раз")
    print(f"  Реальный среднегодовой рост: {real_avg_growth:.1f}%")
    print()

# Визуализация динамики реальных зарплат
plt.figure(figsize=(12, 8))
for sector in salary_data['Отрасль'].unique():
    sector_data = salary_data[salary_data['Отрасль'] == sector]
    plt.plot(sector_data['Год'], sector_data['Реальная_зарплата'], 
             marker='o', linewidth=3, markersize=6, label=sector)

plt.title('Динамика реальных зарплат по отраслям (2000-2024, в ценах 2000 г.)', 
          fontsize=16, fontweight='bold')
plt.xlabel('Год', fontsize=14)
plt.ylabel('Реальная зарплата, руб', fontsize=14)
plt.grid(True, linestyle='--', alpha=0.7)
plt.legend(fontsize=12)
plt.tight_layout()
plt.show()

## 5. Сравнительный анализ

In [ ]:
# Создаем таблицу сравнения роста
growth_comparison = []

for sector in salary_data['Отрасль'].unique():
    sector_data = salary_data[salary_data['Отрасль'] == sector]
    
    # Номинальный рост
    start_salary = sector_data[sector_data['Год'] == 2000]['Зарплата'].iloc[0]
    end_salary = sector_data[sector_data['Год'] == 2024]['Зарплата'].iloc[0]
    nominal_growth = end_salary / start_salary
    
    # Реальный рост
    start_real_salary = sector_data[sector_data['Год'] == 2000]['Реальная_зарплата'].iloc[0]
    end_real_salary = sector_data[sector_data['Год'] == 2024]['Реальная_зарплата'].iloc[0]
    real_growth = end_real_salary / start_real_salary
    
    growth_comparison.append({
        'Отрасль': sector,
        'Зарплата 2000': f"{start_salary:,.0f}",
        'Зарплата 2024': f"{end_salary:,.0f}",
        'Номинальный рост': f"{nominal_growth:.1f}x",
        'Реальная зарплата 2024': f"{end_real_salary:,.0f}",
        'Реальный рост': f"{real_growth:.1f}x"
    })

comparison_df = pd.DataFrame(growth_comparison)
print("=== СРАВНИТЕЛЬНАЯ ТАБЛИЦА РОСТА ЗАРПЛАТ ===")
print()
print(comparison_df.to_string(index=False))

# Визуализация сравнения роста
growth_data = []

for sector in salary_data['Отрасль'].unique():
    sector_data = salary_data[salary_data['Отрасль'] == sector]
    
    start_salary = sector_data[sector_data['Год'] == 2000]['Зарплата'].iloc[0]
    end_salary = sector_data[sector_data['Год'] == 2024]['Зарплата'].iloc[0]
    nominal_growth = end_salary / start_salary
    
    start_real_salary = sector_data[sector_data['Год'] == 2000]['Реальная_зарплата'].iloc[0]
    end_real_salary = sector_data[sector_data['Год'] == 2024]['Реальная_зарплата'].iloc[0]
    real_growth = end_real_salary / start_real_salary
    
    growth_data.append({
        'Отрасль': sector,
        'Номинальный рост': nominal_growth,
        'Реальный рост': real_growth
    })

growth_df = pd.DataFrame(growth_data)

# График сравнения роста
x = np.arange(len(growth_df))
width = 0.35

fig, ax = plt.subplots(figsize=(12, 8))

bars1 = ax.bar(x - width/2, growth_df['Номинальный рост'], width, 
               label='Номинальный рост', alpha=0.8, color='steelblue')
bars2 = ax.bar(x + width/2, growth_df['Реальный рост'], width, 
               label='Реальный рост', alpha=0.8, color='darkorange')

ax.set_xlabel('Отрасль', fontsize=14)
ax.set_ylabel('Рост, раз', fontsize=14)
ax.set_title('Сравнение номинального и реального роста зарплат (2000-2024)', 
             fontsize=16, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(growth_df['Отрасль'])
ax.legend(fontsize=12)
ax.grid(True, linestyle='--', alpha=0.7, axis='y')

# Добавляем значения на столбцы
for bar in bars1:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 0.5,
            f'{height:.1f}x', ha='center', va='bottom', fontsize=10)

for bar in bars2:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height + 0.1,
            f'{height:.1f}x', ha='center', va='bottom', fontsize=10)

plt.tight_layout()
plt.show()

## 6. Ключевые выводы

In [ ]:
print("=== КЛЮЧЕВЫЕ ВЫВОДЫ ИССЛЕДОВАНИЯ ===")
print()
print("1. ЗНАЧИТЕЛЬНЫЙ НОМИНАЛЬНЫЙ РОСТ:")
print("   • Заработные платы во всех анализируемых отраслях выросли в 39-51 раз")
print("   • Это отражает общие инфляционные процессы в российской экономике")
print()
print("2. УМЕРЕННЫЙ РЕАЛЬНЫЙ РОСТ:")
print("   • С учетом инфляции реальное увеличение составило 4-6 раз за 24 года")
print("   • Среднегодовой реальный рост: 6-8%")
print()
print("3. ОТРАСЛЕВАЯ ДИФФЕРЕНЦИАЦИЯ:")
print("   • Образование показало наибольший реальный рост (+523%)")
print("   • Финансовая сфера сохраняет самые высокие абсолютные зарплаты")
print("   • Общая экономика демонстрирует стабильные темпы роста")
print()
print("4. ВЛИЯНИЕ МАКРОЭКОНОМИЧЕСКИХ ШОКОВ:")
print("   • Периоды высокой инфляции (2008, 2014-2015, 2022) снижали реальные доходы")
print("   • Кумулятивная инфляция за период составила 714%")
print()
print("5. ДОЛГОСРОЧНЫЙ ТРЕНД:")
print("   • Несмотря на кризисы, наблюдается устойчивый рост реальных доходов")
print("   • Это свидетельствует о постепенном повышении уровня жизни населения")

## 7. Практическое применение

Результаты исследования имеют важное практическое значение:

**Для работодателей:**
- Планирование зарплатной политики с учетом отраслевых трендов
- Индексация доходов сотрудников на основе реальной динамики
- Сравнительный анализ конкурентоспособности оплаты труда

**Для государственных органов:**
- Оценка эффективности социально-экономической политики
- Планирование бюджетных расходов на оплату труда в госсекторе
- Разработка мер по поддержке отстающих отраслей

**Для исследователей:**
- Методология может применяться для анализа в других странах
- База для дальнейших исследований рынка труда
- Данные для макроэкономического моделирования

## 8. Заключение

Проведенный анализ показал, что за период 2000-2024 годов в России наблюдался значительный номинальный рост зарплат, однако с учетом инфляции реальное увеличение доходов было более умеренным. Образование продемонстрировало наиболее впечатляющую динамику реального роста, что может свидетельствовать о приоритетном внимании государства к социальной сфере.

Важным фактором, влияющим на реальные доходы населения, остается инфляция. Периоды высокой инфляции существенно снижают покупательную способность, подчеркивая важность стабильной монетарной политики для обеспечения роста благосостояния населения.